# <span style="color:#2595bc"><i>K</i>-Armed Bandit Problem</span>
## <span style="color:#2595bc"> Basics and Definitions </span>

Imagine you have a slot machine with $K$-arms or levers. Whenever you pull down one lever, you are given a reward. Each lever arm has a reward associated with it. When the rewards associated with a lever do not change with time, ie they are fixed, it's <span style="color:#f31818"><i>stationary problem</i></span>. When the rewards change with use of levers, it's a <span style="color:#f31818"><i>non-stationary</i></span> one. Your objective is to maximize the expected total reward over some time period, for example, over 1000 action selections, or <i>time steps</i>

Each of the $k$ actions have an expected or mean reward that we expect to get when we select that action. We denote the action selected on time step $t$ as $A_{t}$ and the corresponding reward as $R_{t}$. The <b><span style="color:#f31818">action value</span></b> then of an arbitrary action $a$, denoted $q_{∗}(a)$, is the expected reward given that a is selected:<br>
    $$q_{*} = \mathbb{E}[R_{t}|A_{t}=a]$$
    <br>
Simply put, we <b>estimate</b> how much reward we ought to get when we choose action $a$ based on our previous experiences when we chose the same action.If you knew the value of each action, then it would be trivial to solve the k-armed bandit problem: you would always select the action with highest value in order to maximize the objective. We assume that you do not know the action values with certainty, although we may make estimates as we play the game. Based on our ever-evolving estimates we will choose our option that is most likely to give us the most reward.

## <span style="color:#2595bc"> Explorations VS Exploitation</span>

If you maintain estimates of the action values, then at any time step there is at
least one action whose estimated value is greatest. We call these the <b><i>greedy</i></b> actions.
When you select one of these actions, we say that you are <b><i><span style="color:#f31818">exploiting</i></b></span> your current
knowledge of the values of the actions. If instead you select one of the <b><i>nongreedy</i></b> actions, then we say you are <b><i><span style="color:#f31818">exploring</span></i></b>, because this enables you to improve your
estimate of the nongreedy action’s value. Exploitation is the right thing to do to
maximize the expected reward on the one step, but exploration may produce the
greater total reward in the long run.

Reward is lower in the short run, during exploration, but higher in the long run because after you have discovered the better actions, you can exploit them many times. Because it is not possible both to explore and to exploit with any single action selection, one often refers to the “conflict” between exploration and exploitation.

## <span style="color:#2595bc">Estimating Action Values</span>

The simplest and intuitive way of estimating Action Value at step $t$ for a particular action $a$ is to take the <i>mean</i> of the rewards we got in the past whenever we chose action $a$.<br>
$$Q_{t}(a) = \frac{\text{sum of rewards when $a$ taken prior to $t$}}{\text{number of times $a$ taken prior to $t$}} = \frac{\sum_{i=1}^{t-1}R_{i}\cdot \textbf{1}_{\text A_{i}=a}} {\sum_{i=1}^{t-1}\textbf{1}_{\text A_{i}=a}}$$

where $\textbf 1_{condition}$ denotes is 1 if $condition$ is true and 0 if it is not. Simply put, the above expression just estimates the average returns each time we chose action $a$ , prior to state $t$

If the denominator is zero, then we instead define $Q_{t}(a)$ as some default value,
such as $Q_{1}(a) = 0$

## <span style="color:#2595bc"> Which action to take( or which lever to pull)? </span>



In [2]:
#Import dependencies
import numpy as np
import matplotlib
%matplotlib inline


$$\sum_{n=1}^{\infty} 2^{-n}\cdot A = 1$$
$$k$$
$$\mathbb{E}_{x\in A}$$ 